In [29]:
import pandas as pd

In [35]:
# 출력 옵션 설정
pd.set_option("display.max_rows", None)   # 행 무제한 출력
pd.set_option("display.max_columns", None) # 열 무제한 출력
pd.set_option("display.width", None)      # 너비 무제한
pd.set_option("display.max_colwidth", None) # 컬럼 너비 무제한

In [95]:
# 파일 경로
approval_file = r"C:\Users\apf_temp_admin\Documents\멋사_데이터 분석 5기\프로젝트\2차\data\open\train\train_3승인매출정보.csv"

# chunk 크기
chunk_size = 100000  # 10만 행씩 읽기

# 1️⃣ 첫 chunk만 읽어서 타입과 결측치 확인
approval_reader = pd.read_csv(
    approval_file,
    encoding="utf-8-sig",
    low_memory=False,
    chunksize=chunk_size
)

# 첫 chunk
approval_chunk = next(approval_reader)

print("✅ 승인매출정보 첫 chunk shape:", approval_chunk.shape)

print("\n✅ 데이터 타입")
print(approval_chunk.dtypes)

print("\n✅ 결측치")
print(approval_chunk.isnull().sum())

✅ 승인매출정보 첫 chunk shape: (100000, 407)

✅ 데이터 타입
기준년월                       int64
ID                        object
최종이용일자_기본                  int64
최종이용일자_신판                  int64
최종이용일자_CA                  int64
최종이용일자_카드론                 int64
최종이용일자_체크                  int64
최종이용일자_일시불                 int64
최종이용일자_할부                  int64
이용건수_신용_B0M                int64
이용건수_신판_B0M                int64
이용건수_일시불_B0M               int64
이용건수_할부_B0M                int64
이용건수_할부_유이자_B0M            int64
이용건수_할부_무이자_B0M            int64
이용건수_부분무이자_B0M             int64
이용건수_CA_B0M                int64
이용건수_체크_B0M                int64
이용건수_카드론_B0M               int64
이용금액_일시불_B0M               int64
이용금액_할부_B0M                int64
이용금액_할부_유이자_B0M            int64
이용금액_할부_무이자_B0M            int64
이용금액_부분무이자_B0M             int64
이용금액_CA_B0M                int64
이용금액_체크_B0M                int64
이용금액_카드론_B0M               int64
이용후경과월_신용                  int64
이용후경과월_신판                  i

In [48]:
# 결측치 Series 생성
missing_counts = approval_chunk.isnull().sum()

# 결측치가 0보다 큰 것만 필터링
missing_counts = missing_counts[missing_counts > 0]

# 출력
print("✅ 결측치가 존재하는 컬럼만 출력")
print(missing_counts)

✅ 결측치가 존재하는 컬럼만 출력
_1순위업종            19830
_2순위업종            35278
_3순위업종            43614
_1순위쇼핑업종          35640
_2순위쇼핑업종          46025
_3순위쇼핑업종          53878
_1순위교통업종          46552
_2순위교통업종          67364
_3순위교통업종          83981
_1순위여유업종          74435
_2순위여유업종          91783
_3순위여유업종          97690
_1순위납부업종          49136
_2순위납부업종          83950
_3순위납부업종          96077
최종카드론_금융상환방식코드    81751
최종카드론_신청경로코드      81761
최종카드론_대출일자        82900
dtype: int64


In [55]:
# object 타입 컬럼만 추출
object_cols = approval_chunk.select_dtypes(include="float")

object_cols.columns.to_series().reset_index(drop=True).to_frame(name="Object Columns")

,Object Columns
0,최종카드론_금융상환방식코드
1,최종카드론_대출이율
2,최종카드론_대출일자


In [96]:
for col in approval_chunk.columns:
    unique_vals = pd.Series(approval_chunk[col].unique()).sort_values()
    print(f"🔹 {col} - 유니크 값 ({unique_vals.notna().sum()}개 + NaN 포함):")
    print(unique_vals.values)
    print("-" * 60)

🔹 기준년월 - 유니크 값 (1개 + NaN 포함):
[201807]
------------------------------------------------------------
🔹 ID - 유니크 값 (100000개 + NaN 포함):
['TRAIN_000000' 'TRAIN_000001' 'TRAIN_000002' ... 'TRAIN_099997'
 'TRAIN_099998' 'TRAIN_099999']
------------------------------------------------------------
🔹 최종이용일자_기본 - 유니크 값 (1012개 + NaN 포함):
[   10101 20140823 20150214 ... 20180729 20180730 20180731]
------------------------------------------------------------
🔹 최종이용일자_신판 - 유니크 값 (1009개 + NaN 포함):
[   10101 20150701 20150702 ... 20180729 20180730 20180731]
------------------------------------------------------------
🔹 최종이용일자_CA - 유니크 값 (2611개 + NaN 포함):
[   10101 20080520 20081012 ... 20180728 20180729 20180730]
------------------------------------------------------------
🔹 최종이용일자_카드론 - 유니크 값 (1559개 + NaN 포함):
[   10101 20130628 20131011 ... 20180725 20180726 20180727]
------------------------------------------------------------
🔹 최종이용일자_체크 - 유니크 값 (1887개 + NaN 포함):
[   10101 20120621 20120809 ... 20

In [97]:
summary = []

for col in approval_chunk.columns:
    ser = approval_chunk[col]
    
    # 유니크 개수 (NaN 제외)
    unique_count = ser.dropna().nunique()
    
    # 0 개수
    zero_count = 0
    if pd.api.types.is_numeric_dtype(ser):
        zero_count = (ser == 0).sum()
    
    # NaN 개수
    nan_count = ser.isna().sum()
    
    summary.append({
        "컬럼명": col,
        "유니크값개수": unique_count,
        "0 개수": zero_count,
        "NaN 개수": nan_count
    })

# DataFrame으로 보기 좋게 표시
summary_df = pd.DataFrame(summary)

# 전체 표 출력
from IPython.display import display
display(summary_df)

,컬럼명,유니크값개수,0 개수,NaN 개수
0,기준년월,1,0,0
1,ID,100000,0,0
2,최종이용일자_기본,1012,0,0
3,최종이용일자_신판,1009,0,0
4,최종이용일자_CA,2611,0,0
5,최종이용일자_카드론,1559,0,0
6,최종이용일자_체크,1887,0,0
7,최종이용일자_일시불,922,0,0
8,최종이용일자_할부,1884,0,0
9,이용건수_신용_B0M,93,7580,0


In [101]:
# 파일 경로
approval_file = r"C:\Users\apf_temp_admin\Documents\멋사_데이터 분석 5기\프로젝트\2차\data\open\train\train_4청구입금정보.csv"

# chunk 크기
chunk_size = 100000  # 10만 행씩 읽기

# 1️⃣ 첫 chunk만 읽어서 타입과 결측치 확인
approval_reader = pd.read_csv(
    approval_file,
    encoding="utf-8-sig",
    low_memory=False,
    chunksize=chunk_size
)

# 첫 chunk
approval_chunk = next(approval_reader)

print("✅ 청구입금정보 첫 chunk shape:", approval_chunk.shape)

print("\n✅ 데이터 타입")
print(approval_chunk.dtypes)

print("\n✅ 결측치")
print(approval_chunk.isnull().sum())

✅ 청구입금정보 첫 chunk shape: (100000, 47)

✅ 데이터 타입
기준년월                 int64
ID                  object
대표결제일                int64
대표결제방법코드            object
대표청구지고객주소구분코드       object
대표청구서수령지구분코드        object
청구서수령방법             object
청구서발송여부_B0           int64
청구서발송여부_R3M          int64
청구서발송여부_R6M          int64
청구금액_B0              int64
청구금액_R3M             int64
청구금액_R6M             int64
포인트_마일리지_건별_B0M      int64
포인트_마일리지_건별_R3M      int64
포인트_포인트_건별_B0M       int64
포인트_포인트_건별_R3M       int64
포인트_마일리지_월적립_B0M     int64
포인트_마일리지_월적립_R3M     int64
포인트_포인트_월적립_B0M      int64
포인트_포인트_월적립_R3M      int64
포인트_적립포인트_R12M       int64
포인트_적립포인트_R3M        int64
포인트_이용포인트_R12M       int64
포인트_이용포인트_R3M        int64
포인트_잔여포인트_B0M        int64
마일_적립포인트_R12M        int64
마일_적립포인트_R3M         int64
마일_이용포인트_R12M        int64
마일_이용포인트_R3M         int64
마일_잔여포인트_B0M         int64
할인건수_R3M            object
할인금액_R3M             int64
할인건수_B0M            object
할인금액_B0M             int64
할인금액_청구서

In [58]:
# 결측치 Series 생성
missing_counts = approval_chunk.isnull().sum()

# 결측치가 0보다 큰 것만 필터링
missing_counts = missing_counts[missing_counts > 0]

# 출력
print("✅ 결측치가 존재하는 컬럼만 출력")
print(missing_counts)

✅ 결측치가 존재하는 컬럼만 출력
Series([], dtype: int64)


In [63]:
# object 타입 컬럼만 추출
object_cols = approval_chunk.select_dtypes(include="object")

object_cols.columns.to_series().reset_index(drop=True).to_frame(name="Object Columns")

,Object Columns
0,ID
1,대표결제방법코드
2,대표청구지고객주소구분코드
3,대표청구서수령지구분코드
4,청구서수령방법
5,할인건수_R3M
6,할인건수_B0M
7,source_file


In [103]:
for col in approval_chunk.columns:
    unique_vals = pd.Series(approval_chunk[col].unique()).sort_values()
    print(f"🔹 {col} - 유니크 값 ({unique_vals.notna().sum()}개 + NaN 포함):")
    print(unique_vals.values)
    print("-" * 60)

🔹 기준년월 - 유니크 값 (1개 + NaN 포함):
[201807]
------------------------------------------------------------
🔹 ID - 유니크 값 (100000개 + NaN 포함):
['TRAIN_000000' 'TRAIN_000001' 'TRAIN_000002' ... 'TRAIN_099997'
 'TRAIN_099998' 'TRAIN_099999']
------------------------------------------------------------
🔹 대표결제일 - 유니크 값 (10개 + NaN 포함):
[ 1  5 12 13 15 20 21 23 25 27]
------------------------------------------------------------
🔹 대표결제방법코드 - 유니크 값 (1개 + NaN 포함):
['자동이체']
------------------------------------------------------------
🔹 대표청구지고객주소구분코드 - 유니크 값 (3개 + NaN 포함):
['미확인' '주거지' '회사']
------------------------------------------------------------
🔹 대표청구서수령지구분코드 - 유니크 값 (6개 + NaN 포함):
['K톡명세서+이메일' '당사페이앱+이메일' '문자메세지' '미수신' '우편' '이메일']
------------------------------------------------------------
🔹 청구서수령방법 - 유니크 값 (5개 + NaN 포함):
['K톡' '문자메세지' '미수령' '우편' '이메일']
------------------------------------------------------------
🔹 청구서발송여부_B0 - 유니크 값 (2개 + NaN 포함):
[0 1]
-------------------------------------------

In [104]:
summary = []

for col in approval_chunk.columns:
    ser = approval_chunk[col]
    
    # 유니크 개수 (NaN 제외)
    unique_count = ser.dropna().nunique()
    
    # 0 개수
    zero_count = 0
    if pd.api.types.is_numeric_dtype(ser):
        zero_count = (ser == 0).sum()
    
    # NaN 개수
    nan_count = ser.isna().sum()
    
    summary.append({
        "컬럼명": col,
        "유니크값개수": unique_count,
        "0 개수": zero_count,
        "NaN 개수": nan_count
    })

# DataFrame으로 보기 좋게 표시
summary_df = pd.DataFrame(summary)

# 전체 표 출력
from IPython.display import display
display(summary_df)

,컬럼명,유니크값개수,0 개수,NaN 개수
0,기준년월,1,0,0
1,ID,100000,0,0
2,대표결제일,10,0,0
3,대표결제방법코드,1,0,0
4,대표청구지고객주소구분코드,3,0,0
5,대표청구서수령지구분코드,6,0,0
6,청구서수령방법,5,0,0
7,청구서발송여부_B0,2,22331,0
8,청구서발송여부_R3M,2,17474,0
9,청구서발송여부_R6M,2,14838,0


In [67]:
# 파일 경로
approval_file = r"C:\Users\apf_temp_admin\Documents\멋사_데이터 분석 5기\프로젝트\2차\data\open\train\train_5잔액정보.csv"

# chunk 크기
chunk_size = 100000  # 10만 행씩 읽기

# 1️⃣ 첫 chunk만 읽어서 타입과 결측치 확인
approval_reader = pd.read_csv(
    approval_file,
    encoding="utf-8-sig",
    low_memory=False,
    chunksize=chunk_size
)

# 첫 chunk
approval_chunk = next(approval_reader)

print("✅ 잔액정보 첫 chunk shape:", approval_chunk.shape)

print("\n✅ 데이터 타입")
print(approval_chunk.dtypes)

print("\n✅ 결측치")
print(approval_chunk.isnull().sum())

✅ 잔액정보 첫 chunk shape: (100000, 83)

✅ 데이터 타입
기준년월                   int64
ID                    object
잔액_일시불_B0M             int64
잔액_할부_B0M              int64
잔액_현금서비스_B0M           int64
잔액_리볼빙일시불이월_B0M        int64
잔액_리볼빙CA이월_B0M         int64
잔액_카드론_B0M             int64
월중평잔_일시불_B0M           int64
월중평잔_할부_B0M            int64
월중평잔_CA_B0M            int64
카드론잔액_최종경과월            int64
연체일자_B0M             float64
연체잔액_B0M               int64
연체잔액_일시불_B0M           int64
연체잔액_할부_B0M            int64
연체잔액_현금서비스_B0M         int64
연체잔액_카드론_B0M           int64
연체잔액_대환론_B0M           int64
잔액_현금서비스_B1M           int64
잔액_현금서비스_B2M           int64
잔액_카드론_B1M             int64
잔액_카드론_B2M             int64
잔액_카드론_B3M             int64
잔액_카드론_B4M             int64
잔액_카드론_B5M             int64
잔액_할부_B1M              int64
잔액_할부_B2M              int64
잔액_일시불_B1M             int64
잔액_일시불_B2M             int64
연체일수_B1M               int64
연체일수_B2M               int64
연체원금_B1M               int6

In [68]:
# 결측치 Series 생성
missing_counts = approval_chunk.isnull().sum()

# 결측치가 0보다 큰 것만 필터링
missing_counts = missing_counts[missing_counts > 0]

# 출력
print("✅ 결측치가 존재하는 컬럼만 출력")
print(missing_counts)

✅ 결측치가 존재하는 컬럼만 출력
연체일자_B0M    99562
dtype: int64


In [69]:
# object 타입 컬럼만 추출
object_cols = approval_chunk.select_dtypes(include="object")

object_cols.columns.to_series().reset_index(drop=True).to_frame(name="Object Columns")

,Object Columns
0,ID
1,source_file


In [83]:
sorted_unique = sorted(approval_chunk["연체일자_B0M"].dropna().unique())
print(sorted_unique)

[20180407.0, 20180408.0, 20180409.0, 20180410.0, 20180411.0, 20180412.0, 20180413.0, 20180414.0, 20180415.0, 20180416.0, 20180417.0, 20180418.0, 20180419.0, 20180420.0, 20180421.0, 20180422.0, 20180423.0, 20180424.0, 20180425.0, 20180426.0, 20180427.0, 20180428.0, 20180429.0, 20180430.0, 20180501.0, 20180502.0, 20180503.0, 20180504.0, 20180505.0, 20180506.0, 20180507.0, 20180508.0, 20180509.0, 20180510.0, 20180511.0, 20180512.0, 20180513.0, 20180514.0, 20180515.0, 20180516.0, 20180517.0, 20180518.0, 20180519.0, 20180520.0, 20180521.0, 20180522.0, 20180523.0, 20180524.0, 20180525.0, 20180526.0, 20180527.0, 20180528.0]


In [85]:
for col in approval_chunk.columns:
    unique_vals = pd.Series(approval_chunk[col].unique()).sort_values()
    print(f"🔹 {col} - 유니크 값 ({unique_vals.notna().sum()}개 + NaN 포함):")
    print(unique_vals.values)
    print("-" * 60)

🔹 기준년월 - 유니크 값 (1개 + NaN 포함):
[201807]
------------------------------------------------------------
🔹 ID - 유니크 값 (100000개 + NaN 포함):
['TRAIN_000000' 'TRAIN_000001' 'TRAIN_000002' ... 'TRAIN_099997'
 'TRAIN_099998' 'TRAIN_099999']
------------------------------------------------------------
🔹 잔액_일시불_B0M - 유니크 값 (15624개 + NaN 포함):
[     0      6      7 ... 131214 147188 151961]
------------------------------------------------------------
🔹 잔액_할부_B0M - 유니크 값 (11124개 + NaN 포함):
[    0   481   493 ... 60312 63266 70874]
------------------------------------------------------------
🔹 잔액_현금서비스_B0M - 유니크 값 (4669개 + NaN 포함):
[    0  2992  3284 ... 58985 60143 64059]
------------------------------------------------------------
🔹 잔액_리볼빙일시불이월_B0M - 유니크 값 (2539개 + NaN 포함):
[    0  4328  4370 ... 25116 25407 26181]
------------------------------------------------------------
🔹 잔액_리볼빙CA이월_B0M - 유니크 값 (56개 + NaN 포함):
[    0  5858  5866  5868  5878  5881  5882  5886  5888  5890  5891  5938
  5949 10423 

In [91]:
summary = []

for col in approval_chunk.columns:
    ser = approval_chunk[col]
    
    # 유니크 개수 (NaN 제외)
    unique_count = ser.dropna().nunique()
    
    # 0 개수
    zero_count = 0
    if pd.api.types.is_numeric_dtype(ser):
        zero_count = (ser == 0).sum()
    
    # NaN 개수
    nan_count = ser.isna().sum()
    
    summary.append({
        "컬럼명": col,
        "유니크값개수": unique_count,
        "0 개수": zero_count,
        "NaN 개수": nan_count
    })

# DataFrame으로 보기 좋게 표시
summary_df = pd.DataFrame(summary)

# 전체 표 출력
from IPython.display import display
display(summary_df)

,컬럼명,유니크값개수,0 개수,NaN 개수
0,기준년월,1,0,0
1,ID,100000,0,0
2,잔액_일시불_B0M,15624,27383,0
3,잔액_할부_B0M,11124,64577,0
4,잔액_현금서비스_B0M,4669,94635,0
5,잔액_리볼빙일시불이월_B0M,2539,97218,0
6,잔액_리볼빙CA이월_B0M,56,99942,0
7,잔액_카드론_B0M,5902,93809,0
8,월중평잔_일시불_B0M,16920,26330,0
9,월중평잔_할부_B0M,12834,63156,0
